In [ ]:
from sumeval.metrics.rouge import RougeCalculator
from nubia_score.nubia import Nubia

nubia = Nubia()
rouge = RougeCalculator(stopwords=True, lang="en")

In [ ]:
nlp_sm = spacy.load("en_core_web_sm")
nlp_md = en_core_web_md.load()

In [ ]:
result = []
all_scores_df =[ ]
all_labels = []

select_features = True
valid = True


for i in range(1):

  print(i)
  number_text = 10
  xml = format_xml(str(introductions[number_text]))
  text = format_text(str(introductions[number_text]), post_processing=False)
  reference = format_text(str(abstract[number_text]), post_processing=True)

  bibs = get_citations(xml)
  text = exclude_bib(text, bibs)
  text = format_text(text, post_processing=True)

  soup = BeautifulSoup(text)
  text = soup.get_text()

  sentences = split_sentences([text])
  sentences = list(map(str, sentences[0]))
  sentences = format_sentences(sentences)

  try: 
    if select_features:
      features = main(sentences, xml, article_keyword, nlp, nlp_st)
      features_df = format_df (sentences, features)
      result.append(features_df)

    if valid:
      sentences_ref = split_sentences([reference])
      sentences_ref = list(map(str, sentences_ref[0]))
      scores_df, label = main_create_label(sentences, sentences_ref, rouge, nubia)
      scores_df['label'] = label

      all_scores_df.append(scores_df)
      all_labels.append(label)

  except ValueError as error:
      print(repr(error))
      pass

In [ ]:
for i in range(len(result)):
  df = pd.DataFrame(result[i])
  df.to_csv("features_df/df_{}.csv".format(i), index=False)

In [ ]:
for i in range(len(all_scores_df)):
  df = pd.DataFrame(all_scores_df[i])
  df.to_csv("features_df/score_df_{}.csv".format(i), index=False)